In [1]:
#https://github.com/MapleFrogStudio/algo_vibes_SP500_RSI
#https://stackoverflow.com/questions/38647052/use-pyevolve-in-python-3
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np
from datetime import datetime, date

In [2]:
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Mutators
from pyevolve import Initializators
from pyevolve import Selectors
from pyevolve import GAllele
from pyevolve import Consts

In [3]:
# def grab_symbols():
#     # Grab S&P Symbols from Wikipedia or local HTML File
#     # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#     # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
#     tickers = pd.read_html('./tickers.html')[0]
#     tickers = tickers.Symbol.to_list()
#     tickers = [i.replace('.','-') for i in tickers]
#     return tickers 

# print('Grab symbols function defined...')

In [3]:
def yahoo_prices1(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    years_back = 3
    end_dt = datetime.strptime(current_day, '%Y-%m-%d').date()
    start_dt = date(end_dt.year - years_back, end_dt.month, end_dt.day)

    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')

Yahoo_prices function defined...


In [4]:
def yahoo_prices(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    #years_back = 3
    start_dt = '2020-01-01' #date(end_dt.year - years_back, end_dt.month, end_dt.day)
    end_dt = '2023-01-31'#datetime.strptime(current_day, '%Y-%m-%d').date()
    
    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')

Yahoo_prices function defined...


In [5]:
# Grab symbols and get prices from yahoo
tickers = ['AAPL','MSFT','GOOGL']#,'SPY']#,'BAC','TJX','AMD']#grab_symbols()
test_length = len(tickers)
all_assets = yahoo_prices(tickers[0:test_length], '2023-01-31')

[*********************100%***********************]  3 of 3 completed


In [6]:
all_assets.head()

Symbol   Adj Close       Close        High         Low        Open  \
Date                                                                            
2019-12-31   AAPL   71.810936   73.412498   73.419998   72.379997   72.482498   
2019-12-31  GOOGL   66.969498   66.969498   67.032997   66.606499   66.789497   
2019-12-31   MSFT  152.930145  157.699997  157.770004  156.449997  156.770004   
2020-01-02   AAPL   73.449394   75.087502   75.150002   73.797501   74.059998   
2020-01-02  GOOGL   68.433998   68.433998   68.433998   67.324501   67.420502   

               Volume  
Date                   
2019-12-31  100805600  
2019-12-31   19514000  
2019-12-31   18369400  
2020-01-02  135480400  
2020-01-02   27278000

In [7]:
all_assets.isnull().sum()

Symbol       0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

In [8]:
# Generate our indicators for Buy Signals for each of our assets
# df: contains one company (symbol) of data
#def add_indicators(df,chromosome[0],chromosome[1]):
def add_indicators(df,a,b):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    #print('c0',a)
    df['NextOpen'] = df.Open.shift(-1)
    df['NextDate'] = df.index
    df['NextDate'] = df.NextDate.shift(-1)
    df['MA'] = df['Close'].rolling(window=a).mean()
    
    df['Price_change'] = df['Close'].pct_change()
    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=b).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=b).mean()
    
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))

    # df.loc[(df['Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy_signal' ] = 'Yes'
    # df.loc[(df['Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'
    #print(df['RSI'])
    return df
    
print('add_RSI_buy_signals function defined....')

add_RSI_buy_signals function defined....


In [9]:
def add_buy_signals(df,c):
    #print('rsi', c)
    buy_filter = (df['Close'] > df['MA']) & (df['RSI'] < c)
    df.loc[buy_filter, 'Buy_signal' ] = 'Yes'
    df.loc[(df['Close'] <= df['MA']) | (df['RSI'] >= c), 'Buy_signal' ] = 'No'


    return df

def add_sell_signals(df):
    
    pass

def generate_orders(df):
    pass


In [10]:
def order_simulator(df,d,e,f):
    max_hold_days = d   # hold position for a maximum of 10 days
    stop_loss_percentage = e * -0.02
    target_percentage = f * 0.02
    # Order actions : Enter, Skip, Close, Stop, Target (a buy signal will be skipped if already in position)
    df['Order'] = ''

    in_position_start_index = None
    for index, row in df.iterrows():
        # Check for Enter position conditions
        if row.Buy_signal == 'Yes' and in_position_start_index is None:
            df.at[index, 'Order'] = 'Enter'
            in_position_start_index = index
        elif row.Buy_signal == 'Yes' and in_position_start_index is not None:
            df.at[index, 'Order'] = 'Skip'
        
        # Check for exit position conditions
        if in_position_start_index is not None:
            if (index - in_position_start_index).days >= max_hold_days:
                df.at[index, 'Order'] = 'Close'
                in_position_start_index = None
            elif index > in_position_start_index: 
                buy_price = df.at[in_position_start_index, 'Close']
                current_price = row['Close']
                percent_change = (current_price - buy_price) / buy_price
                if percent_change < stop_loss_percentage:
                    df.at[index, 'Order'] = 'Stop'
                    in_position_start_index = None
                if percent_change > target_percentage:
                    df.at[index, 'Order'] = 'Target'
                    in_position_start_index = None
    
    return df

print('Order simulator function defined....')

Order simulator function defined....


In [11]:
def extract_trades(df):
    # Loop through frame order information
    ## Grab Enter row to add to our trade dict
    ## Grab Sell row (Close, Target, Stop) to add to our ENTER row
    ## append to our trades
    ## Convert to a DataFrame
    # trade = {Id:999, Trade_seq: Tade_seq, Symbol: Ticker, Buy_dt: Date, Buy_Price: Price, Sell_dt: Date, Sell_Price: Price, Sell_Trigger: Close }
    # Id = symbol_Sort
    
    trades = []
    open_position = False
    trade_sequence = 1
    for index, row in df.iterrows():
        if not open_position: # Get the next Entry row
            if row.Order == 'Enter':
                trade_seq = trade_sequence
                symbol = row.Symbol
                buy_dt = row.NextDate
                buy_price = row.NextOpen
                open_position = True
        else: # Get the next Sell row (Close, Target, Stop)
            if row.Order in ['Close','Target','Stop']:
                sell_dt = row.NextDate
                sell_price = row.NextOpen
                sell_type = row.Order
                days_in_trade = (sell_dt - buy_dt).days
                profit = sell_price - buy_price
                percent_return = profit / buy_price
                trade = {'Id':f'{symbol}-{trade_seq}','Trade_seq':trade_seq,'Symbol':symbol,\
                         'Buy_dt':buy_dt,'Buy_Price':buy_price,\
                         'Sell_dt':sell_dt,'Sell_Price':sell_price,'Sell_type':sell_type,\
                         'Duration':days_in_trade,'profit':profit,'return':percent_return\
                        }
                trades.append(trade)
                open_position = False
                trade_sequence += 1
    trades_df = pd.DataFrame(trades)
    #print('profit ',trades_df.profit)
    #print(trades_df)
    return trades_df

print('Extract Trades function defined....')  

Extract Trades function defined....


In [12]:
# add indicators and buy signals, generate orders and store detailed results by date and symbol in an easily accessible list
def ma_mean_reversion_1(chromosome):
    
    multi_assets = []
    len_tickers = len(tickers[0:test_length])
    
    for i in range(len_tickers):
        frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
        frame = add_indicators(frame,chromosome[0],chromosome[1])
        frame = add_buy_signals(frame,chromosome[2])
        if frame is not None:
            frame = order_simulator(frame,chromosome[3],chromosome[4],chromosome[5])
            multi_assets.append(frame)

    all_trades = None
    for i in range(len(multi_assets)):
        frame = multi_assets[i]
        trades = extract_trades(frame)
        #print(trades)
        if all_trades is None:
            all_trades = trades
        else:    
            all_trades = pd.concat([all_trades, trades], axis=0, ignore_index=True)


    wining_trades = len(all_trades.loc[all_trades.profit > 0])
    losing_trades = len(all_trades.loc[all_trades.profit < 0])
    target_trades = len(all_trades.loc[all_trades.Sell_type == 'Target'])
    stop_trades   = len(all_trades.loc[all_trades.Sell_type == 'Stop'])
    close_trades  = len(all_trades.loc[all_trades.Sell_type == 'Close'])
#     print(f'Wining trades: {wining_trades}')
#     print(f'Losing trades: {losing_trades}')
#     print(f'Targets: {target_trades}, Stops: {stop_trades}, Closes: {close_trades}')
#     print(f'Total profit : ${all_trades.profit.sum():0.2f}')
    #print(all_trades)
    r1 = all_trades['return']/0.02
    sqn = (r1.mean()/r1.std()) * np.sqrt(r1.size)
    
    #if trades.profit.sum() < 0:
    if sqn < 0:
        score = 0.0001
    else:
        score = sqn #trades.profit.sum()
        
    #print('Score: ',score)
    return score

In [14]:
# # add indicators and buy signals, generate orders and store detailed results by date and symbol in an easily accessible list
# def ma_mean_reversion_1(chromosome):
    
#     multi_assets = []
#     len_tickers = len(tickers[0:test_length])
#     for i in range(len_tickers):
#         frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
#         frame = add_indicators(frame,chromosome)
#         frame = add_buy_signals(frame,chromosome)
#         if frame is not None:
#             frame = order_simulator(frame,chromosome)
#             multi_assets.append(frame)

#     all_trades = None
#     for i in range(len(multi_assets)):
#         frame = multi_assets[i]
#         trades = extract_trades(frame)
#         if all_trades is None:
#             all_trades = trades
#         else:    
#             all_trades = pd.concat([all_trades, trades], axis=0, ignore_index=True)


#     wining_trades = len(all_trades.loc[all_trades.profit > 0])
#     losing_trades = len(all_trades.loc[all_trades.profit < 0])
#     target_trades = len(all_trades.loc[all_trades.Sell_type == 'Target'])
#     stop_trades   = len(all_trades.loc[all_trades.Sell_type == 'Stop'])
#     close_trades  = len(all_trades.loc[all_trades.Sell_type == 'Close'])
#     print(f'Wining trades: {wining_trades}')
#     print(f'Losing trades: {losing_trades}')
#     print(f'Targets: {target_trades}, Stops: {stop_trades}, Closes: {close_trades}')
#     print(f'Total profit : ${all_trades.profit.sum():0.2f}')
#     #all_trades
#     score = all_trades.profit.sum()
#     print('Profit: ',score)
#     return score

In [13]:
setOfAlleles = GAllele.GAlleles()
              
#Chromosome 0 > MA
ma = GAllele.GAlleleRange(10, 20, real=False)
setOfAlleles.add(ma)

#Chromosome 1 >EWM
ewm = GAllele.GAlleleRange(7,14, real=False) #eurusd; usdchf;eurjpy,audusd;gbpjpy;usdcad;eurcad
setOfAlleles.add(ewm)

#Chromosome 2 >RSI
rsi = GAllele.GAlleleRange(50,60, real=False)
setOfAlleles.add(rsi)

#Chromosome 3 > Hold
hold = GAllele.GAlleleRange(5, 10, real=False)
#sloss = GAllele.GAlleleRange(1.0, 1.5, real=True)
setOfAlleles.add(hold)

#Chromosome 4 > stop loss
#periods = GAllele.GAlleleRange(5, 10, real=False)
sloss = GAllele.GAlleleRange(1,1.5, real=True)
setOfAlleles.add(sloss)

#Chromosome 5 > target
#periods = GAllele.GAlleleRange(5, 10, real=False)
targ = GAllele.GAlleleRange(1.5,2.5, real=True)
setOfAlleles.add(targ)

genome = G1DList.G1DList(6)
genome.setParams(allele=setOfAlleles)

# The evaluator function (objective function)
genome.evaluator.set(ma_mean_reversion_1)

# This mutator and initializator will take care of
   # initializing valid individuals based on the allele set
   # that we have defined before
genome.mutator.set(Mutators.G1DListMutatorAllele)
genome.initializator.set(Initializators.G1DListInitializatorAllele)
  
   # Genetic Algorithm Instance
ga = GSimpleGA.GSimpleGA(genome)
ga.minimax = Consts.minimaxType["maximize"]
#ga.setMinimax(Consts.minimaxType["minimize"])
ga.selector.set(Selectors.GRouletteWheel)
ga.setGenerations(100)
ga.setPopulationSize(100)
  
   # Do the evolution, with stats dump
   # frequency of 10 generations
ga.evolve(freq_stats=1)
  
   # Best individual
best = ga.bestIndividual()
print ("\nBest individual score: %.10f" % (best.score,))
print (best)

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw)             [1.29(2.64)/0.93(0.00)/1.07(1.07)]
Gen. 1 (1.00%): Max/Min/Avg Fitness(Raw)             [1.42(2.64)/0.99(0.00)/1.18(1.18)]
Gen. 2 (2.00%): Max/Min/Avg Fitness(Raw)             [1.51(2.64)/1.04(0.09)/1.26(1.26)]
Gen. 3 (3.00%): Max/Min/Avg Fitness(Raw)             [1.61(2.64)/1.08(0.08)/1.34(1.34)]
Gen. 4 (4.00%): Max/Min/Avg Fitness(Raw)             [1.63(2.64)/1.09(0.12)/1.36(1.36)]
Gen. 5 (5.00%): Max/Min/Avg Fitness(Raw)             [1.68(2.64)/1.13(0.22)/1.40(1.40)]
Gen. 6 (6.00%): Max/Min/Avg Fitness(Raw)             [1.67(2.69)/1.12(0.14)/1.40(1.40)]
Gen. 7 (7.00%): Max/Min/Avg Fitness(Raw)             [1.68(3.03)/1.21(0.29)/1.40(1.40)]
Gen. 8 (8.00%): Max/Min/Avg Fitness(Raw)             [1.60(3.07)/1.13(0.00)/1.34(1.34)]
Gen. 9 (9.00%): Max/Min/Avg Fitness(Raw)             [1.68(3.07)/1.18(0.10)/1.40(1.40)]
Gen. 10 (10.00%): Max/Min/Avg Fitness(Raw)             [1.65(3.07)/1.15(0.00)/1.38(1.38)]
Gen. 11 (11.00%): Max/Min/Avg 

In [25]:
np.sqrt(r1.size)

5.385164807134504